In [3]:
import torch
from torch import nn
from music21 import stream, note, duration

In [4]:
class LSTMGenerator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMGenerator, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(2, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(2, batch_size, self.hidden_size).to(x.device)
        out, _ = self.lstm(x.unsqueeze(1), (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def generate_music(generator, num_notes=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator.to(device)
    generator.eval()

    # Generate random input for the generator
    input_noise = torch.randn(num_notes, 100).to(device)

    # Generate output from the generator
    with torch.no_grad():
        generated_notes = generator(input_noise).cpu().numpy()

    # Convert the generated notes into a music stream
    music_stream = stream.Stream()

    for note_data in generated_notes:
        pitch = int(note_data[0])  # Assuming the first value represents the pitch
        duration_value = float(note_data[1])  # Assuming the second value represents the duration
        offset_value = float(note_data[2])  # Assuming the third value represents the offset

        new_note = note.Note(pitch)
        new_note.duration = duration.Duration(duration_value)
        new_note.offset = offset_value

        music_stream.append(new_note)

    return music_stream

In [8]:
# Load the trained generator model
generator = LSTMGenerator(input_size=100, hidden_size=256, output_size=3, num_layers=2)
generator.load_state_dict(torch.load('models/generator_loss_0.1853.pth'))
generator.eval()

# Generate music using the trained generator model
generated_music = generate_music(generator, num_notes=100)

# Save the generated music as a MIDI file
generated_music.write('midi', fp='generated_music.mid')

print("Generated music saved as 'generated_music.mid'")


Generated music saved as 'generated_music.mid'
